<a href="https://colab.research.google.com/github/sofiafranken/ce/blob/main/agente_itau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU pypdf
%pip install -U langchain
%pip install -U langchain-community
%pip install -U langchain-groq
%pip install langchain-huggingface
%pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
from google.colab import userdata
import os
api_key=userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = api_key

TimeoutException: Requesting secret GROQ_API_KEY timed out. Secrets can only be fetched when running from the Colab UI.

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model = 'llama-3.3-70b-versatile')
#llm = ChatGroq(model='deepseek-r1-distill-llama-70b')

In [ ]:
prompt = '''
messages: 'Segundo o guia do Itaú, quais sao os principais desafios e preocupacoes relacionadas ao uso da inteligencia artificial?'
'''

In [ ]:
resposta = llm.invoke(prompt)

In [ ]:
print(f'Resposta gerada: {resposta.text}')

In [ ]:
!gdown --id 18p-ZpN4B6u6hBlZbRbv8Qe7X3azDB4gM -O guia.pdf

In [ ]:
url = 'O guia.pdf'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

url = 'guia.pdf'
loader = PyPDFLoader(url)
pages = []

for page in loader.load_and_split():
  pages.append(page)

In [ ]:
print(f'Metadados: {pages[0].metadata}\n')

In [ ]:
print(f'Resposta: {pages[0].page_content}')

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [ ]:
vector_store =InMemoryVectorStore.from_documents(pages, embed_model)

In [ ]:
docs = vector_store.similarity_search("Objectives Development Process", k=2)

In [ ]:
for doc in docs:
  print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]}\n')

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
template = '''
You´re helpful assistent that only gives answers bases on the given context.
If the answer is not in the context, say "I don´t know".

Context: {context}
Question: {question}
Answer:

'''

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = chain.invoke("What are the Objectives Development Process?")
print(f'Response: {response}')